# Language model

Language model is a probability distribution over sequences of word.
In this lab we will apply laguage model for a classification problem. The task is to implement a filter for spam documents.

### Dataset
Download this https://www.kaggle.com/uciml/sms-spam-collection-dataset dataset.
Here we normalize the text and split it by sentences using nltk library. Then sentences are splitted to the terms. For simplicity we will lose the punctuation and characters register.

In [20]:
import pandas as pd
import re
import string
import nltk
import random
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn import metrics

In [21]:
def normalize(text):
    text = text.lower()
    text = re.sub('\W+',' ', text)
    text = text.encode("ascii", errors="ignore").decode()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

def tokenize(text):
    sentences = sent_tokenize(text)
    words = []
    for sent in sentences:
        words.append(word_tokenize(normalize(sent)))
    return words                

In [74]:
messages = pd.read_csv('spam.csv', encoding="latin1")
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["category", "text"]
display(messages.head(n = 10))

spam_messages = messages[messages["category"] == "spam"]["text"]
ham_messages = messages[messages["category"] == "ham"]["text"]

spam_sentences = []
ham_sentences = [] 

for message in spam_messages:
    spam_sentences.extend(tokenize(message))
    
for message in ham_messages:
    ham_sentences.extend(tokenize(message)) 

#list of sentences, each sentence represented as a list of terms

,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


Calculate the average length and average number of sentences in spam message.

In [23]:
message_count = len(spam_messages)
message_length = 0
sentence_number = 0
sentence_length = 0

for spam in spam_messages:
    sentences = sent_tokenize(spam)
    for sent in sentences:
        sentence_length += len(word_tokenize(sent))
    sentence_number += len(sentences)
    message_length += len(word_tokenize(spam))
    
spam_message_length = round(message_length/message_count)
spam_message_sentence_length = round(sentence_length/sentence_number)
spam_message_sentence_number = round(sentence_number/message_count)

print('average length of a spam message: ', message_length/message_count)
print('average length of a sentence in spam message: ', sentence_length/sentence_number)
print('average number of sentences in spam message: ', sentence_number/message_count)

average length of a spam message:  27.70682730923695
average length of a sentence in spam message:  9.174202127659575
average number of sentences in spam message:  3.0200803212851404


### Unigram model

Calculate the number of occurancies of each term separately for spam and ham messages and the total number of terms.

In [79]:
def build_dict(sentences):
    terms = dict()
    for sent in sentences:
        for word in sent:
            count = terms.get(word, 0)
            terms.update({word: count + 1})
    return terms

In [25]:
spam_term_c = build_dict(spam_sentences)
spam_N = len(spam_term_c)
ham_term_c = build_dict(ham_sentences)
ham_N = len(ham_term_c)

print(spam_N, ham_N)

2886 6851


Print 10 most popular words in spam messages.

In [27]:
spam_sorted = sorted(spam_term_c.items(), key=lambda x: x[1], reverse=True)

print(*spam_sorted[:10])

('to', 688) ('a', 378) ('call', 355) ('you', 297) ('your', 264) ('free', 224) ('2', 206) ('the', 206) ('for', 204) ('now', 199)


### Bigram model

We use sentence begining and sentence ending as a special terms ("<s" and "/s>" respectively). 

Calculate the number of occuracnies for bigrams (as a key in dictionary we use words, separated by the space symbol) - **build_bigram_dict**.

Also, for using in a generative model, for each term we will need a list of next term, found in the dataset - **build_next_term**.

In [84]:
def build_bigram_dict(sentences):
    terms = dict()
    for sent in sentences:
        sent.insert(0, '<s') 
        sent.append('/s>')
        for i in range(len(sent) - 1):
            word1 = sent[i]
            word2 = sent[i+1]
            bigram = word1 + ' ' + word2
            count = terms.get(bigram, 0)
            terms.update({bigram: count + 1})
            
    return terms

def build_next_term(spam_dict, spam_bigram_dict):
    next_term = dict()
    for word in spam_dict:
        terms = []
        for bigram in spam_bigram_dict:
            if bigram.startswith(word+" "):
                length = len(word) + 1
                next_word = bigram[length:]
                count = spam_bigram_dict[bigram]
                terms.append((next_word, count))
        terms = sorted(terms, key=lambda x: x[1], reverse=True)
        next_term[word] = terms
    return next_term 

In [85]:
spam_term_c["<s"] = len(spam_sentences)
spam_term_c["/s>"] = len(spam_sentences)

ham_term_c["<s"] = len(ham_sentences)
ham_term_c["/s>"] = len(ham_sentences)

spam_bigram_c = build_bigram_dict(spam_sentences)
spam_next_words = build_next_term(spam_term_c, spam_bigram_c)

ham_bigram_c = build_bigram_dict(ham_sentences)

In [86]:
print(spam_next_words["will"])

[('be', 25), ('b', 4), ('blow', 2), ('go', 2), ('not', 1), ('u', 1), ('recieve', 1), ('receive', 1), ('arrive', 1), ('the', 1), ('smith', 1), ('/s>', 1), ('ignore', 1), ('enter', 1), ('give', 1)]


Which bigrams are the most popular in spam messages?

From which words spam sentence usually begins?

In [87]:
spam_bigram_sorted = sorted(spam_bigram_c.items(), key=lambda x: x[1], reverse=True)

print("most popular bigrams in spam messages are:")
print(*spam_bigram_sorted[:10], "\n")

print("most popular starting words in spam messages are:")
startings = [(bigram[0][3:], bigram[1]) for bigram in spam_bigram_sorted if '<s' in bigram[0]] 
print(*startings[:10])

most popular bigrams in spam messages are:
('<s <s', 2256) ('/s> /s>', 2256) ('<s call', 149) ('now /s>', 98) ('<s you', 83) ('you have', 73) ('<s your', 67) ('<s to', 66) ('<s txt', 61) ('have won', 54) 

most popular starting words in spam messages are:
('<s', 2256) ('call', 149) ('you', 83) ('your', 67) ('to', 66) ('txt', 61) ('urgent', 53) ('free', 39) ('text', 39) ('this', 37)


### Genetative model

Now using out language model let's generate a spam message. 

We use the calculated values for average sentence number in message and an average sentence length.
Just start with a random sentence beginning ```random.choice(spam_next_words['<s'])[0]``` and go through dictionary of bigrams, choosing randomly next word of ```randomness``` most popular to follow.

In [88]:
def generate_spam(start_word="", randomness = 5):
    message = ""
    for _ in range(spam_message_sentence_number):
        if len(start_word) == 0:
            start_word = random.choice(spam_next_words['<s'])[0]
        message += start_word[:1].upper() + start_word[1:]
        for _ in range(spam_message_sentence_length):
            next_possible = spam_next_words[start_word]
            next_word = random.choice(next_possible[:min(randomness, len(next_possible))])[0]
            
            if next_word == "/s>":
                message += ". "
                break
                
            message += " " + next_word
            start_word = next_word  
        if not next_word == "/s>":
            message += ". "
        start_word = ""
    return message

In [97]:
print(generate_spam())

I have 1 new mobiles with u. Hard live operator. Pushbutton 2 contact u u know u can win our. 


In [302]:
print(generate_spam())

Hungry gay chat to claim call now. Wow. Claire here http www ldew com1win150ppmx3age16. 


In [397]:
print(generate_spam("urgent"))

Urgent message. Call 08000930705 for free entry into 121 chat to 86688. Text messages by a 2000 gift. 


In [346]:
print(generate_spam("hello"))

Hello from your mobile update for a new voicemail. This message waiting for free entry std text back if. This week. 


### Smoothing

The problem is that if the bigram $t_1 t_2$ occuted $0$ times in the corpus, the conditional probability $P(t_2|t_1) = 0$

The solution is smoothing. Read this document https://nlp.stanford.edu/~wcmac/papers/20050421-smoothing-tutorial.pdf

Here we implement the Jelinek-Mercer smoothing (interpolation) and a functions (smoothing_prob_spam & smoothing_prob_ham), that return the conditional probability $P(t_2 | t_1)$ with a smoothing.

In [98]:
def smoothing_prob_spam(x, s, l=0.8):
    if s + " " + x in spam_bigram_c:
        prob_xs = spam_bigram_c[s + " " + x] / spam_term_c[s]
    else: 
        prob_xs = 0
    if x in spam_term_c:
        prob_x = spam_term_c[x] / sum(spam_term_c.values())
    else: 
        prob_x = 0
    return l * prob_xs + (1 - l) * prob_x

def smoothing_prob_ham(x, s, l=0.8):
    if s + " " + x in ham_bigram_c:
        prob_xs = ham_bigram_c[s + " " + x] / ham_term_c[s]
    else: 
        prob_xs = 0
    if x in ham_term_c:
        prob_x = ham_term_c[x] / sum(ham_term_c.values())
    else: 
        prob_x = 0
    return l * prob_xs + (1 - l) * prob_x

### Classification

Now, let's implement a Bayesian Classifier for the sentence and test one of our generated sentences on it.

The Bayesian Classifier returns, which probability is higher: $P(spam|t_1, \dots , t_k)$ or $P(ham|t_1, \dots , t_k)$, where:

$$P(spam|t_1, \dots , t_k) = \frac{P(t_1, \dots , t_k|spam)P(spam)}{P(t_1, \dots , t_k)} \sim P(t_1, \dots , t_k|spam)P(spam)$$ 
$$\sim P(t_1 | BEGIN, spam) \cdot \sim P(t_2 | t_1, spam) \cdot \dots \cdot \sim P(END | t_k, spam)$$

(and the same for ham)

In [99]:
def classify(sentence):
    sentence = word_tokenize(normalize(sentence))
    sentence.insert(0, '<s') 
    sentence.append('/s>')
    prob_spam = 1
    prob_ham = 1
    for i in range(len(sentence) - 1):
        word1 = sentence[i]
        word2 = sentence[i+1]
        prob_spam = prob_spam * smoothing_prob_spam(word2, word1)
        prob_ham = prob_ham * smoothing_prob_ham(word2, word1)
    if prob_spam > prob_ham: return 1
    return 0

In [101]:
spam = classify("Hello from your mobile update for a new voicemail")
if spam: print("the sentence is spam")
else: print("the sentence is ham")

the sentence is spam


### Testing
Let's now evaluate the classifier using sklearn metrics

In [102]:
spam_sentences_raw = []
for message in spam_messages:
    spam_sentences_raw.extend(sent_tokenize(message))
    
ham_sentences_raw = []
for message in ham_messages:
    ham_sentences_raw.extend(sent_tokenize(message))


X = spam_sentences_raw + ham_sentences_raw
y = [1] * len(spam_sentences_raw) + [0] * len(ham_sentences_raw)
y_pred = []
for sentence in X:
    y_pred.append(classify(sentence))

In [103]:
print("Accuracy:",metrics.accuracy_score(y, y_pred))

Accuracy: 0.9957646210687573


In [104]:
print(metrics.classification_report(y, y_pred))
print("0: ham")
print("1: spam")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8841
           1       0.98      0.99      0.99      2256

    accuracy                           1.00     11097
   macro avg       0.99      1.00      0.99     11097
weighted avg       1.00      1.00      1.00     11097

0: ham
1: spam
